### LLM as a Judge

As métricas apresentadas até agora tem uma coisa em comum: todas elas são exatas, no sentido em que contabilizam a existência de tokens/palavras no texto avaliado para obter um score. Mesmo no caso dos testes baseados em MMLU, existe uma opção válida (a,b,c,d,etc), o que facilita uma validação.
O que dizer então de respostas abertas, muito comuns em chatbots por exemplo?
Neste caso, entra um conceito chamado de Judging LLM-as-a-Judge.
Em se tratando de assuntos subjetivos, a idéia é utilizar um modelo superior para atuar como "árbitro", como GPT-4 por exemplo.
Além disso, existe um benchmark chamado MT-BENCH, que consiste de questões abertas criadas para avaliar a capacidade conversacional e habilidade para seguir instruções. E claro, um processo de criação dessas questões e respostas, utilizando humanos, é ideal para avaliar preferências, porém muito trabalhoso e lento. 
Uma outro possibilidade também é utilizar the Judge Model para avaliar as respostas, e coletar também um feedback humano sobre a decisão. Seria uma maneira de medir o quanto existe de concordância entre o humano e o modelo agindo como Juiz.
O assunto vai mais longe ainda, se considerarmos que pode existir viés, embutido tanto nas respostas do modelo, quanto nas respostas de um humano. 
Existem diversas formas de executar uma avaliação:
Comparação de pares de respostas: qual a melhor resposta, dada a pergunta xyz e um determinado critério
Avaliação de uma resposta, baseada em um critério
Avaliação baseada em referências: a resposta é avaliada com base em critérios e também com base em outras referências, como por exemplo, conteúdo proveniente de RAG

Entre as vantagens de um LLM como Juiz, podemos considerar a escabilidade, que se traduz na redução do envolvimento humano no processo (apesar que ele deverá se envolver em alguma medida, ou você delegaria todo o processo para o LLM?) e a explicabilidade, que realmente é muito importante, em se tratando de assuntos subjetivos.



In [8]:
#! pip install langchain_community
#! pip install oci

In [41]:
import oci
import json

#Bibliotecas para uso do langchain + OCI Generative AI
from langchain_community.chat_models import ChatOCIGenAI
from langchain_community.chat_models import oci_generative_ai


### Autenticação OCI

In [42]:
COMPARTMENT_ID = "ocid1.compartment.oc1..aaaaaaaacuafyhpnjnsp5luoo3dqklsdi2ysobswq3irzu664gl3cjhvcjpa"
AUTH_TYPE = "API_KEY" # The authentication type to use, e.g., API_KEY (default), SECURITY_TOKEN, INSTANCE_PRINCIPAL, RESOURCE_PRINCIPAL.

# Auth Config
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('config', CONFIG_PROFILE)

### Instância OCI Generative AI com modelo gpt 4.1 nano, executado dentro do OCI

In [43]:
# Service endpoint
#Atenção ao endpoint, porque alguns modelos estão disponíveis em regiões especificas. 
endpoint = "https://inference.generativeai.us-ashburn-1.oci.oraclecloud.com"

In [44]:
llm_oci = ChatOCIGenAI(
    model_id="ocid1.generativeaimodel.oc1.iad.amaaaaaask7dceyacxqiaijwxalbynhst6oyg4wttzagz3dai4y2rnwh6wrq", #open ai gpt 4.1 nano
    service_endpoint=endpoint,
    compartment_id=COMPARTMENT_ID,
    provider="meta",
    model_kwargs={
        "temperature": 0.1,
        "max_tokens": 600,
        #"frequency_penalty": 0,
        #"presence_penalty": 0,
        "top_p": 0.75,
        "top_k": None#
        #"seed": None
    },
    auth_type=AUTH_TYPE,
    auth_profile="DEFAULT",
    auth_file_location='config'
)

### Avaliação

Este teste utiliza um llm as a judge para avaliar 3 diferentes respostas candidatas, que serão avaliadas por um modelo gpt 4.1 nano. O modelo fornecerá score, justificativa e deve citar eventuais problemas com a resposta. No final, deve salvar a resposta em formato json.

In [45]:

reference = """
Explique como adicionar elementos a uma lista em Python.
"""

candidates = {
    "A": "Use lista.append(x) para adicionar um item no final.",
    "B": "Você pode usar append ou extend. Também é possível usar insert para colocar em uma posição específica.",
    "C": "Para adicionar itens em Python use lista.push(x)."
}

# --- Monta o bloco das candidatas ---
candidates_block = "\n".join([f"{k}: {v}" for k, v in candidates.items()])

# --- Prompt de juiz para o LLM ---
judge_prompt = f"""
Você é um avaliador especializado e imparcial. 

Sua tarefa é avaliar cada resposta candidata com base em:
- Correção factual
- Completude
- Clareza
- Adequação ao enunciado
- Uso correto da API de Python

Retorne a resposta no seguinte formato:
[
  {{"candidate": "A", "score": 0.0 a 1.0, "justification": "texto curto", "issues": "issues"}},
]

Precisa finalizar corretamente o json
Enunciado de referência:
{reference}

Respostas candidatas:
{candidates_block}
"""


In [46]:
response = llm_oci.invoke([judge_prompt])
resposta = response.content.strip().lower()

print("\n--- Raw JSON output do juiz ---\n")
print(resposta)

# Converte para Python
print("\n--- Como objeto Python ---\n")
judgments = json.loads(resposta)
print(judgments)


--- Raw JSON output do juiz ---

[
  {
    "candidate": "a",
    "score": 1.0,
    "justification": "resposta correta, clara e completa, usando a api adequada.",
    "issues": ""
  },
  {
    "candidate": "b",
    "score": 0.9,
    "justification": "resposta correta e completa, incluindo métodos adicionais, clara e adequada ao enunciado.",
    "issues": ""
  },
  {
    "candidate": "c",
    "score": 0.0,
    "justification": "incorreta, pois não existe o método 'push' em listas python.",
    "issues": "uso incorreto do método 'push', que não faz parte da api de listas em python."
  }
]

--- Como objeto Python ---

[{'candidate': 'a', 'score': 1.0, 'justification': 'resposta correta, clara e completa, usando a api adequada.', 'issues': ''}, {'candidate': 'b', 'score': 0.9, 'justification': 'resposta correta e completa, incluindo métodos adicionais, clara e adequada ao enunciado.', 'issues': ''}, {'candidate': 'c', 'score': 0.0, 'justification': "incorreta, pois não existe o método 'pus